In [1]:
from assembly_info_utils import fetch_all_assembly_data, create_assembly_info_df, fetch_all_statuses
import pandas as pd
import sys
# Укажи путь к папке, где лежит utils_warehouse.py
sys.path.append(r'D:\Pytnon_scripts\warehouse_scripts')
sys.path.append(r'D:\Pytnon_scripts\tokens.json')
from utils_warehouse import load_api_tokens, create_insert_table_db

In [26]:
# Получаем данные по сборочным заданиям
all_data = await fetch_all_assembly_data()
# Создаем датафрейм с информацие по сборочным заданиям
df_assembly_id = create_assembly_info_df(all_data)
# Создаем словарь для запроса статусов сборочных заданий
assembly_dict = df_assembly_id.groupby('account')['id'].apply(list).to_dict()
# Получаем информацию по сборочным заданиям в даатфрейме
df_statuses = await fetch_all_statuses(assembly_dict, load_api_tokens())
# Объединяем датафреймы по сборочным заданиям и их статусам
df_status_model = pd.merge(df_assembly_id, df_statuses, how='left', on=['id', 'account'])

Получены данные по кабинету Вектор
Получены данные по кабинету Тоноян
Получены данные по кабинету Оганесян
Получены данные по кабинету Даниелян
Получены данные по кабинету Лопатина
Получены данные по кабинету Пилосян
Получены данные по кабинету Старт
Получены данные по кабинету Вектор
Получены данные по кабинету Лопатина
Получены данные по кабинету Тоноян
Получены данные по кабинету Оганесян
Получены данные по кабинету Даниелян
Получены данные по кабинету Пилосян
Получены данные по кабинету Вектор
Получены данные по кабинету Старт
Получены данные по кабинету Лопатина
Получены данные по кабинету Тоноян
Получены данные по кабинету Оганесян
Получены данные по кабинету Даниелян
Получены данные по кабинету Лопатина
Получены данные по кабинету Хачатрян
Получены данные по кабинету Пилосян
Получены данные по кабинету Вектор
Получены данные по кабинету Тоноян
Получены данные по кабинету Оганесян
Получены данные по кабинету Старт
Получены данные по кабинету Хачатрян
Получены данные по кабинету Т

In [27]:
df_status_model.columns

Index(['address', 'scanPrice', 'comment', 'deliveryType', 'supplyId',
       'orderUid', 'article', 'colorCode', 'rid', 'createdAt', 'offices',
       'skus', 'id', 'warehouseId', 'nmId', 'chrtId', 'price',
       'convertedPrice', 'currencyCode', 'convertedCurrencyCode', 'cargoType',
       'isZeroOrder', 'options', 'officeId', 'account', 'createdAt_msk',
       'local_vendor_code', 'supplierStatus', 'wbStatus'],
      dtype='object')

In [28]:
# Переименуем для корректного перемещения в БД
df_status_model = df_status_model.rename(columns={'scanPrice' : 'scan_price',
                                                  'deliveryType' : 'delivery_type',
                                                  'orderUid' : 'order_uid',
                                                  'colorCode' : 'color_code',
                                                  'createdAt' : 'created_at',
                                                  'warehouseId' : 'warehouse_id',
                                                  'chrtId' : 'chrt_id',
                                                  'convertedPrice' : 'converted_price',
                                                  'currencyCode' : 'currency_code',
                                                  'convertedCurrencyCode' : 'converted_currency_code',
                                                  'cargoType' : 'cargo_type',
                                                  'isZeroOrder' : 'is_zero_order',
                                                  'officeId' : 'office_id',
                                                  'createdAt_msk' : 'created_at_msk',
                                                  'nmId': 'nm_id', 
                                                  'article' : 'vendor_code',
                                                  'supplyId' : 'supply_id',
                                                  'supplierStatus' : 'supplier_status',
                                                  'wbStatus' : 'wb_status'})

In [29]:
df_status_model['date'] = pd.to_datetime(df_status_model['created_at_msk']).dt.date

In [30]:
df_status_model = df_status_model[['date', 'nm_id', 'local_vendor_code', 'vendor_code', 'id', 'supplier_status', 'wb_status',
                                    'supply_id', 'address', 'scan_price', 'price', 'converted_price', 'comment', 'delivery_type',
                                    'order_uid', 'color_code', 'rid', 'created_at', 'created_at_msk', 'offices', 'skus',
                                    'warehouse_id', 'chrt_id', 'currency_code', 'converted_currency_code', 'cargo_type', 'is_zero_order',
                                    'options', 'office_id', 'account']]
df_status_model.head()

,date,nm_id,local_vendor_code,vendor_code,id,supplier_status,wb_status,supply_id,address,scan_price,...,skus,warehouse_id,chrt_id,currency_code,converted_currency_code,cargo_type,is_zero_order,options,office_id,account
0,2025-07-23,468245116,wild1545,wild1545d,3654026978,complete,sold,WB-GI-170267206,None,0.0,...,[2044882837437],715584,657912296,643,643,1,False,{'isB2B': False},242,Даниелян
1,2025-07-23,152033635,wild252,wild252,3654027233,complete,sold,WB-GI-170267206,None,0.0,...,[2037609343137],715584,254549475,643,643,1,False,{'isB2B': False},242,Даниелян
2,2025-07-23,155893983,wild273,wild273,3654047261,complete,sold,WB-GI-170267206,None,0.0,...,[2037697672942],715584,260148750,643,643,1,False,{'isB2B': False},242,Даниелян
3,2025-07-23,151948248,wild164,wild164,3654058970,complete,sold,WB-GI-170267206,None,0.0,...,[2037607301504],715584,254420921,933,643,1,False,{'isB2B': False},242,Даниелян
4,2025-07-23,156056295,wild160,wild160D,3654061257,complete,sold,WB-GI-170267206,None,0.0,...,[2037701988779],715584,260397248,643,643,1,False,{'isB2B': False},242,Даниелян


In [31]:
# Поменяем порядок колонок
# df_status_model = df_status_model[[]]
df_status_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115786 entries, 0 to 115785
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype                        
---  ------                   --------------   -----                        
 0   date                     115786 non-null  object                       
 1   nm_id                    115786 non-null  int64                        
 2   local_vendor_code        115786 non-null  object                       
 3   vendor_code              115786 non-null  object                       
 4   id                       115786 non-null  int64                        
 5   supplier_status          115786 non-null  object                       
 6   wb_status                115786 non-null  object                       
 7   supply_id                115786 non-null  object                       
 8   address                  0 non-null       object                       
 9   scan_price               107688 non-n

In [ ]:
# На всякий случай, проверяем наличие дубликатов
if df_status_model[['id', 'supplier_status', 'wb_status']].duplicated().sum() > 0:
    df_status_model = df_status_model.drop_duplicates(subset=['id', 'supplier_status', 'wb_status'])
# Устанавливаем типы данных для БД
columns_type = {
    'date': 'DATE',  # или 'DATE', если это дата в формате '2025-08-21'
    'nm_id': 'BIGINT',
    'local_vendor_code': 'VARCHAR(50)',
    'vendor_code': 'VARCHAR(50)',
    'id': 'BIGINT',
    'supplier_status': 'VARCHAR(50)',
    'wb_status': 'VARCHAR(50)',
    'supply_id': 'VARCHAR(50)',
    'address': 'TEXT',
    'scan_price': 'DOUBLE PRECISION',
    'price': 'INTEGER',
    'converted_price': 'INTEGER',
    'comment': 'TEXT',
    'delivery_type': 'VARCHAR(50)',
    'order_uid': 'TEXT',
    'color_code': 'VARCHAR(50)',
    'rid': 'TEXT',
    'created_at': 'TIMESTAMPTZ',  # TIMESTAMP WITH TIME ZONE
    'created_at_msk': 'VARCHAR(50)',
    'offices': 'JSONB',
    'skus': 'JSONB',
    'warehouse_id': 'INTEGER',
    'chrt_id': 'BIGINT',
    'currency_code': 'INTEGER',
    'converted_currency_code': 'INTEGER',
    'cargo_type': 'INTEGER',
    'is_zero_order': 'BOOLEAN',
    'options': 'JSONB',
    'office_id': 'INTEGER',
    'account': 'VARCHAR(50)'
}
# Задаем имя таблицы
table_name = 'assembly_task_status_model'
# Задаем уникальные поля для проверки уникальности данных
key_cols = ('id', 'supplier_status', 'wb_status')
# Отправляем данные в таблицу БД
create_insert_table_db(df_status_model, table_name, columns_type, key_cols)

Соединение с базой данных установлено.
Таблица assembly_task_status_model создана или уже существует.
Данные успешно добавлены в БД.
Соединение с базой данных закрыто.
